# <center>Phase 2 - Publication<center>

In [2]:
import sqlite3
import pandas as pd
from matplotlib_venn import venn2, venn3
import scipy.stats as scs
import textwrap
import matplotlib.pyplot as plt
import numpy as np
import seaborn as sns
from  itertools import combinations
import os
from matplotlib.colors import ListedColormap
from matplotlib import ticker
from scipy.stats import ttest_ind

# IPython magics for this notebook.
%matplotlib inline

# Use latex font for matplotlib
plt.rc('text', usetex=True)
plt.rc('font', family='serif')

In [3]:
class FieldRegistry:
    
    def __init__(self):
        self.fields = {}
        self.labels = {}
    
    def addToRegistry(self, index, labels, fields, category):
        self.labels[category] = {}
        for counter, value in enumerate(labels, 1):
            key = '{}{:02}'.format(index, counter)
            self.labels[category][key] = value
        self.fields[category] = fields
        
FR = FieldRegistry()   
CATEGORY = ['Aggression', 'Fear/Anxiety', 'Compulsion', 'House Soiling', 'Excessive Barking', 
            'Jumping', 'Mounting/Humping', 'Coprophagia', 'Destructive Behavior',
            'Rolling in Repulsive Material', 'Running Away/Escaping', 
            'Overactivity/Hyperactivity']
        
# Aggression.
labels = ['Familiar people in the home', 'Stangers visiting the home',
          'Stangers away from the home', 'Another dog in the home',
          'Unfamiliar dogs visiting the home', 'Unfamiliar dogs on walks (off lead)',
          'Unfamiliar dogs on walks (on lead)', 'Veterinarians', 'Trainers', 'Groomers',
          'Animals other than dogs in the home']
fields = ['q03_main_1', 'q03_main_2', 'q03_main_3', 'q03_main_4', 'q03_main_5', 'q03_main_6',
          'q03_main_7', 'q03_main_8', 'q03_main_9', 'q03_main_10', 'q03_main_11']
FR.addToRegistry('A', labels, fields, CATEGORY[0])

# Fear/anxiety.
labels = ['Thunderstorm phobia', 'Noise phobia', 'Crowd phobia', 'Phobia of other dogs',
          'PTSD', 'Generalized anxiety', 'Situational anxiety', 'Veterinarian phobia',
          'Separation anxiety', 'Travel anxiety', 'Other']
fields = ['q04_1', 'q04_2', 'q04_3', 'q04_4', 'q04_5', 'q04_6', 'q04_7', 'q04_8', 'q04_9',
          'q04_10', 'q04_11']
FR.addToRegistry('B', labels, fields, CATEGORY[1])

# Compulsion.
labels = ['Spinning', 'Tail chasing', 'Shadow/light chasing', 'Running in geometric patterns',
          'Licking of wrist/hock', 'Fly snapping', 'Sucking flank region/blankets',
          'Tennis ball fetish', 'Collecting/arranging objects', 'Nail biting',
          'Digging in yard', 'Stone/rock chewing', 'Other']
fields = ['q05_main_1', 'q05_main_2', 'q05_main_3', 'q05_main_4', 'q05_main_5', 'q05_main_6',
          'q05_main_7', 'q05_main_8', 'q05_main_9', 'q05_main_10', 'q05_main_11',
          'q05_main_12', 'q05_main_13']
FR.addToRegistry('C', labels, fields, CATEGORY[2])

# House soiling.
## soil_type = ['Urine', 'Feces', 'Urine and feces']
labels = ['Urine', 'Feces', 'Urine and feces', 'Specific locations', 'Anywhere',
          'Owner present', 'Owner away', 'Excited/overwhelmed']
fields = ['q06_soil_type_1', 'q06_soil_type_2', 'q06_soil_type_3','q06_soil_location_1',
          'q06_soil_location_2', 'q06_situation_1', 'q06_situation_2', 'q06_situation_3']
FR.addToRegistry('D', labels, fields, CATEGORY[3])

# Excessive barking.
labels = ['Owner present', 'Owner away', 'To get attention', 'At tiggers (inside)',
          'At triggers (outside)', 'During car rides']
fields = ['q07_sitatuon_1', 'q07_sitatuon_2', 'q07_sitatuon_3', 'q07_sitatuon_4',
          'q07_sitatuon_5', 'q07_sitatuon_6']
FR.addToRegistry('E', labels, fields, CATEGORY[4])

# Jumping.
labels = ['Owner', 'Family members', 'Strangers']
fields = ['q08_who_1', 'q08_who_2', 'q08_who_3']
FR.addToRegistry('F', labels, fields, CATEGORY[5])

# Mounting/humping.
labels = ['People', 'Familiar dogs', 'Unfamiliar dogs', 'Inanimate objects']
fields = ['q09_main_1', 'q09_main_2', 'q09_main_3', 'q09_main_4']
FR.addToRegistry('G', labels, fields, CATEGORY[6])

# Consuming feces.
labels = ['Their own', "Other dogs'", "Other species'"]
fields = ['q10_main_1', 'q10_main_2', 'q10_main_3']
FR.addToRegistry('H', labels, fields, CATEGORY[7])

# Destructive behavior.
labels = ['Owner is home', 'Owner is away']
fields = ['q11_situation_1', 'q11_situation_2']
FR.addToRegistry('I', labels, fields, CATEGORY[8])

# Rolling in repulsive materials.
labels = ['Urine', 'Feces', 'Dead Stuff', 'Garbage']
fields = ['q12_main_1', 'q12_main_2', 'q12_main_3', 'q12_main_4']
FR.addToRegistry('J', labels, fields, CATEGORY[9])

# Running away/escaping.
labels = ['Escapes when out', 'Escapes from home', 'Escapes from confinement',
          'Returns home after escape']
fields = ['q14_out', 'q14_house', 'q14_conf', 'q14_return']
FR.addToRegistry('K', labels, fields, CATEGORY[10])

# Overactivity/hyperactivity.
labels = ['Constant moving/jumping', 'Difficulty settling', 'Highly distractible',
          'Impulsive']
fields = ['q15_main_1', 'q15_main_2', 'q15_main_3', 'q15_main_4']
FR.addToRegistry('L', labels, fields, CATEGORY[11])

In [4]:
# Data Globals
SIG_P = 0.01
TOTAL_USERS = 0
REMAINING_USERS = 0
TOTAL_DOGS = 0
REMAINING_DOGS = 0
PREVALENCE = lambda x: (x / REMAINING_DOGS) * 100
CATEGORY_MATRIX = pd.DataFrame()
QUESTION_MATRIX = pd.DataFrame()

# Database Globals
USER_TABLE = 'users'
DOG_TABLE = 'dogs'
BIAS_FILTER = '''
    USING (record_id)
    WHERE question_reason_for_part_3 = 0
    OR (question_reason_for_part_3 = 1 AND q01_main != 1)'''
CON = sqlite3.connect('../data/processed/processed.db')

In [5]:
def createStringDataFrame(table, fields, labels, filtered=True):
    query = 'SELECT ' + fields + ' FROM ' + table
    if filtered:
        table2 = USER_TABLE if table == DOG_TABLE else DOG_TABLE
        query += ' JOIN ' + table2 + ' ' + BIAS_FILTER
    df = pd.read_sql_query(query, CON)
    df.columns = labels
    return df

def convertToNumeric(df):
    df = df.apply(pd.to_numeric, errors='coerce')
    return df

def createNumericDataFrame(table, fields, labels, filtered=True):
    df = createStringDataFrame(table, fields, labels, filtered)
    return convertToNumeric(df)

def replaceFields(df, column, replacement_dict):
    df[column].replace(replacement_dict, inplace=True)

def getValueCountAndPrevalence(df, field):
    s = df[field].value_counts()
    p = s.apply(PREVALENCE)
    rv = pd.concat([s, p], axis=1)
    rv.columns = ['frequency', 'prevalence']
    return rv

def checkSignificance(p, sig=SIG_P):
    if p < sig:
        print('The resulting p-value is below the set significance threshold (%.2f).' %sig)
        
def exportTable(data, title):
    file_ = os.path.join('..', 'reports', 'tables', title) + '.tex'
    with open(file_, 'w') as tf:
        tf.write(r'\documentclass[varwidth=\maxdimen]{standalone}\usepackage{booktabs}\begin{document}')
        tf.write(df.to_latex())
        tf.write(r'\end{document}')
        
def exportFigure(figure, title):
    file_ = os.path.join('..', 'reports', 'figures', title) + '.png'
    figure.tight_layout()
    figure.savefig(file_, dpi = 500)

def createCategoryMatrix():
    fields = []
    labels = []
    counter = 1
    for cat, subdict in FR.labels.items():
        for key, value in  subdict.items():
            if counter == 11:
                counter += 1;
            fields.append('q02_main_{}'.format(counter))
            labels.append(key[0])
            break
        counter += 1
    fields = ', '.join(fields)
    df = createNumericDataFrame(DOG_TABLE, fields, labels, filtered=True)
    cols = []
    pvalue = {}
    for col in df:
        cols.append(col)
        pvalue[col] = {}
    pairs = list(combinations(df.columns, 2))
    for pair in pairs:
        contingency = pd.crosstab(df[pair[0]], df[pair[1]])
        c, p, dof, expected = scs.chi2_contingency(contingency, correction=False)
        pvalue[pair[0]][pair[1]] = p
        pvalue[pair[1]][pair[0]] = p
    df = pd.DataFrame(pvalue).sort_index(ascending=True)
    return df

def createQuestionMatrix():
    fields = ''
    for cat, sublist in FR.fields.items():
        for field in sublist:
            fields += '{}, '.format(field)
    fields = fields[:-2]
    labels = []
    for cat, subdict in FR.labels.items():
        for key, value in  subdict.items():
            labels.append(key)
    df = createNumericDataFrame(DOG_TABLE, fields, labels, filtered=True)
    cols = []
    pvalue = {}
    for col in df:
        cols.append(col)
        pvalue[col] = {}
    pairs = list(combinations(df.columns, 2))
    for pair in pairs:
        contingency = pd.crosstab(df[pair[0]], df[pair[1]])
        c, p, dof, expected = scs.chi2_contingency(contingency, correction=False)
        pvalue[pair[0]][pair[1]] = p
        pvalue[pair[1]][pair[0]] = p
    df = pd.DataFrame(pvalue).sort_index(ascending=True)
    return df

def createCorrelationMatrix():
    fields = []
    labels = []
    counter = 1
    for cat, subdict in FR.labels.items():
        for key, value in  subdict.items():
            if counter == 11:
                counter += 1;
            fields.append('q02_main_{}'.format(counter))
            labels.append(key[0])
            break
        counter += 1
    fields = ', '.join(fields)
    df = createNumericDataFrame(DOG_TABLE, fields, labels, filtered=True)
    return df.corr()

def createOddsRatioMatrix():
    fields = []
    labels = []
    counter = 1
    for cat, subdict in FR.labels.items():
        for key, value in  subdict.items():
            if counter == 11:
                counter += 1;
            fields.append('q02_main_{}'.format(counter))
            labels.append(key[0])
            break
        counter += 1
    fields = ', '.join(fields)
    df = createNumericDataFrame(DOG_TABLE, fields, labels, filtered=True)
    cols = []
    pvalue = {}
    for col in df:
        cols.append(col)
        pvalue[col] = {}
    pairs = list(combinations(df.columns, 2))
    for pair in pairs:
        contingency = pd.crosstab(df[pair[0]], df[pair[1]])
        c, p, dof, expected = scs.chi2_contingency(contingency, correction=False)
        pvalue[pair[0]][pair[1]] = getOddsRatio(contingency)
        pvalue[pair[1]][pair[0]] = getOddsRatio(contingency)
    df = pd.DataFrame(pvalue).sort_index(ascending=True)
    return df

def displayOddsRatio(df):
    odds = getOddsRatio(df)
    print("odds ratio = %f" %odds)  

def getOddsRatio(df):
    return (df[1][1]/df[1][0])/(df[0][1]/df[0][0])

def get_significance_category(p):
    if np.isnan(p):
        return p
    elif p > 10**(-3):
        return -1
    elif p <= 10**(-3) and p > 10**(-6):
        return 0
    else:
        return 1 

## <center>Demographics</center>

### Number of participants:

In [15]:
df = createNumericDataFrame(USER_TABLE, 'COUNT(*)', ['count'], filtered=False)
# Assign value to global.
TOTAL_USERS = df['count'][0]
print('N = %d owners [unadjusted]' %TOTAL_USERS)

N = 3201 owners (unadjusted)


### Number of participating dogs:

In [16]:
df = createNumericDataFrame(DOG_TABLE, 'COUNT(*)', ['count'], filtered=False)
# Assign value to global.
TOTAL_DOGS = df['count'][0]
print('N = %d dogs [unadjusted]' %TOTAL_DOGS)

N = 5018 dogs (unadjusted)


### Motivation for enrollment:

In [12]:
# Create the necessary dataframe.
fields = ('question_reason_for_part_1, question_reason_for_part_2, '
          'question_reason_for_part_3, question_reason_for_part_4, '
          'question_reason_for_part_5')
labels = ['Love for dogs', 'You help shelter animals', 'Suspicion of behavior problems',
          'Work with animals', 'Other']
df = createNumericDataFrame(USER_TABLE, fields, labels, filtered=False)
s = df.sum(0, skipna=False)
s = s.sort_values(ascending=False)
df = pd.DataFrame(index=s.index, data={'Count':s.values})
df.columns.name = 'Motivation'
display(df)
# NOTE: This does not include a count of users who did not provide a reason.

Motivation,Count
Love for dogs,2595
You help shelter animals,1377
Work with animals,1049
Suspicion of behavior problems,830
Other,273


### Suspicion of behavior problems as one of multiple motivating factors:

In [24]:
# Create the necessary dataframe.
fields = ('question_reason_for_part_1, question_reason_for_part_2, '
          'question_reason_for_part_3, question_reason_for_part_4, '
          'question_reason_for_part_5')
labels = ['love for dogs', 'you help shelter animals', 'suspicion of behavior problems',
          'work with animals', 'other']
df = createNumericDataFrame(USER_TABLE, fields, labels, filtered=False)
df = df[df[labels[2]] == 1]
df['sum'] = df.sum(axis=1)
s = df.sum(0, skipna=False)

print('n = %d owners (%d%%) [unadjusted]' %(s.iloc[2], (s.iloc[2]/TOTAL_USERS * 100)))

n = 830 owners (25%) [unadjusted]


### Suspicion of behavior problems as the sole motivating factor:

In [22]:
# Create the necessary dataframe.
fields = ('question_reason_for_part_1, question_reason_for_part_2, '
          'question_reason_for_part_3, question_reason_for_part_4, '
          'question_reason_for_part_5')
labels = ['love for dogs', 'you help shelter animals', 'suspicion of behavior problems',
          'work with animals', 'other']
df = createNumericDataFrame(USER_TABLE, fields, labels, filtered=False)
df = df[df[labels[2]] == 1]
df['sum'] = df.sum(axis=1)
df = df[df['sum'] == 1]
s = df.sum(0, skipna=False)

print('n = %d owners (%d%%) [unadjusted]' %(s.iloc[2], (s.iloc[2]/TOTAL_USERS * 100)))

n = 98 owners (3%) [unadjusted]


### Adjusting sample for bias:

In [20]:
fields = 'q02_score'
labels = ['Score']

df_gross = createNumericDataFrame(DOG_TABLE, fields, labels, filtered=False)
cnt_total_dogs_w_problems = len(df_gross[df_gross[labels[0]] != 0].index)

df_adjusted_dogs = createNumericDataFrame(DOG_TABLE, fields, labels)
REMAINING_DOGS = len(df_adjusted_dogs.index)
cnt_total_dogs_w_problems_adjusted = len(df_adjusted_dogs[df_adjusted_dogs[labels[0]] != 0].index)

df_adjusted_users = createNumericDataFrame(USER_TABLE, 'COUNT(DISTINCT email)', ['count'])
REMAINING_USERS = df_adjusted_users['count'][0]


# Display the count results.
print('\nAdjusted study population:')
print('N = %d owners (adjusted)' %REMAINING_USERS)
print('N = %d dogs (adjusted)' %REMAINING_DOGS)


Adjusted study population:
N = 2480 owners (adjusted)
N = 4114 dogs (adjusted)


### Dogs per household:

In [15]:
fields = 'record_id'
labels = ['record index']
df = createStringDataFrame(DOG_TABLE, fields, labels)

record_dict = {}
for index, row in df.iterrows():
    key = row.iloc[0]
    if not key in record_dict:
        record_dict[key] = 1
    else:
        record_dict[key] += 1

s = pd.Series(record_dict, name='dogs')
df = pd.DataFrame(index=s.index, data={'dogs':s.values})
display(df.describe().round(2))

,dogs
count,2480.00
mean,1.66
std,0.96
min,1.00
25%,1.00
50%,1.00
75%,2.00
max,13.00


### Age at date of response

In [ ]:
fields = 'dog_age_today_months'
labels = ['age (months)']
df = createNumericDataFrame(DOG_TABLE, fields, labels)
display(df.describe())

### Age at neutering:

In [ ]:
fields = 'dog_sex_month'
labels = ['age (months)']
df = createNumericDataFrame(DOG_TABLE, fields, labels)
display(df.describe())

### Number of dogs per source of origin:

In [ ]:
# Construct the dataframe.
fields = 'acquisition_source'
labels = ['origin']
df = createStringDataFrame(DOG_TABLE, fields, labels)
# Rescue fields (online: 1, in-person: 4) were combined in the database.
replacements = {'':'no response', '1': 'rescue', '2': 'online (non-rescue)', '3': 'pet store', '5': 'breeder',
                '6': 'self-bred', '7': 'friends/family', '8': 'found', '9': 'other'}
replaceFields(df, labels[0], replacements)
df = df.apply(pd.value_counts)
df.rename(index=str, columns={labels[0]: "frequency"}, inplace=True)
df.columns.name = labels[0]
display(df)

### Age at onset:

In [ ]:
fields = 'q01_age_months'
labels = ['age (months)']
df = createNumericDataFrame(DOG_TABLE, fields, labels)
display(df.describe())

fields = 'q01_acq'
labels = ['evident when acquired']
df = createNumericDataFrame(DOG_TABLE, fields, labels)
cnt_evident_when_acquired = len(df[df[labels[0]] == 1].index)
print('Number of users who reported that the behavior problem was evident when acquired: %d'
      %(cnt_evident_when_acquired))

### Gender:

In [ ]:
# Construct the dataframe.
fields = 'dog_sex'
labels = ['gender']
df = createStringDataFrame(DOG_TABLE, fields, labels)
replacements = {'':'no response', '1':'male', '2':'female'}
replaceFields(df, labels[0], replacements)
df = df.apply(pd.value_counts)
df.rename(index=str, columns={labels[0]: "frequency"}, inplace=True)
df.columns.name = labels[0]
display(df)

### Neutered status:

In [ ]:
# status: unverified
# Construct the dataframe.
fields = 'dog_sex, dog_spayed'
labels = ['Gender', 'Neutered']
df = createStringDataFrame(DOG_TABLE, fields, labels)
replacements = {'':'No response', '1':'Male', '2':'Female'}
replaceFields(df, labels[0], replacements)
replacements = {'':'No response', '0':'No', '1':'Yes', '2':"I don't know"}
replaceFields(df, labels[1], replacements)
display(pd.crosstab(df[labels[0]], df[labels[1]], margins=True))

### Number of purebred dogs:

In [ ]:
# status: verified
fields = 'purebred'
labels = ['purebred']
df = createStringDataFrame(DOG_TABLE, fields, labels)
df.head()
replacements = {'':'No response', '0':'No', '1':'Yes'}
replaceFields(df, labels[0], replacements)
df = df.apply(pd.value_counts)
df.rename(index=str, columns={labels[0]: "frequency"}, inplace=True)
df.columns.name = labels[0]
display(df)

### Number of purebred dogs without a breed designated:

In [ ]:
# status: verified
# Create the necessary dataframe.
fields = 'purebred_breed, purebred'
labels = ['breed', 'purebred']
df = createStringDataFrame(DOG_TABLE, fields, labels)
df = df[df[labels[1]] == '1']
df.drop(columns=labels[1], inplace=True)
df = df[df[labels[0]] == '']
purebred_missing_breed = df.describe().iloc[0][0]
print('Number of purebreds without breed designated: %d dogs' %(purebred_missing_breed))

### Number of purebred breeds:

In [ ]:
# status: verified
# Create the necessary dataframe.
fields = 'purebred_breed, purebred'
labels = ['breed', 'purebred']
df = createStringDataFrame(DOG_TABLE, fields, labels)
df = df[df[labels[1]] == '1']
df.drop(columns=labels[1], inplace=True)
df = df[df[labels[0]] != '']
display(df.describe())

### Numbers of dogs per purebred breed:

In [ ]:
# status: verified
# Create the necessary dataframe.
fields = 'purebred_breed, purebred'
labels = ['breed', 'purebred']
df = createStringDataFrame(DOG_TABLE, fields, labels)
df = df[df[labels[1]] == '1']
df.drop(columns=labels[1], inplace=True)
df = df[df[labels[0]] != '']

# Calculate sums and prevalences.
df = getValueCountAndPrevalence(df, labels[0])
df = df.round(2)
df.columns.name = labels[0]
display(df[:30])

## <center>Prevalence of Behavior Problems</center>

In [ ]:
print('Dogs with behavior problems: n = %d (%d%%)'
      %(cnt_total_dogs_w_problems_adjusted,
       (cnt_total_dogs_w_problems_adjusted/REMAINING_DOGS * 100)))

### Prevalence of overall behavior problems:

In [ ]:
# Calculate the gross prevalence.
prevalence_gross = (cnt_total_dogs_w_problems / TOTAL_DOGS) * 100

# Calculate the adjusted prevalence.
prevalence_adjusted = PREVALENCE(cnt_total_dogs_w_problems_adjusted)

# Calculate the difference between the gross and adjusted prevalence.
diff_prevalence = prevalence_adjusted - prevalence_gross
sign = '+' if diff_prevalence > 0 else ''

# Display the prevalence results.
print('Pre-adjustment prevalence: %.2f%% (%d/%d dogs)'
      %(prevalence_gross, cnt_total_dogs_w_problems, TOTAL_DOGS))
print('Post-adjustment prevalence: %.2f%% (%d/%d dogs)'
      %(prevalence_adjusted, cnt_total_dogs_w_problems_adjusted, REMAINING_DOGS))
print('Shift in prevalence as a result of adjustment: %s%.2f%%'
      %(sign, diff_prevalence))

### Prevalence of behavior problem categories (Table 1)

In [ ]:
# Create the necessary dataframe.
fields = []
labels = []
for counter, category in enumerate(CATEGORY, 1):
    if counter > 10:
        counter += 1;
    fields.append('q02_main_{}'.format(counter))
    labels.append(category)
fields = ', '.join(fields)
df = createNumericDataFrame(DOG_TABLE, fields, labels, filtered=True)

# Get individual behavior problem counts and display as a table.
sums = df.sum()
sums = sums.sort_values(ascending=False)

# Calculate the prevalence of each behavior problem.
total_dogs = len(df.index)
prevalences = sums.apply(PREVALENCE)

# Create a table.
df = pd.DataFrame(index=sums.index, data={'Count':sums.values,
                                          'Prevalence (%)': prevalences.values.round(2)})
df.columns.name = 'Category'
display(df)
exportTable(df, 'table_1')

### Prevalence of behavior problem category subtypes (Table 2)

In [ ]:
# Create the necessary dataframe.
sums = pd.Series()
for i in range(0, 12):
    all_fields = FR.fields[CATEGORY[i]].copy()
    all_labels = list(FR.labels[CATEGORY[i]].values()).copy()
    df = createNumericDataFrame(DOG_TABLE, ', '.join(all_fields), all_labels, filtered=True)
    if sums.empty:
        sums = df.sum().sort_values(ascending=False)
    else:
        sums = sums.append(df.sum().sort_values(ascending=False))

# Calculate the prevalence of each behavior problem.
prevalences = sums.apply(PREVALENCE)

# Create a table.
df = pd.DataFrame(index=sums.index, data={'Frequency':sums.values,
                                          'Prevalence (%)': prevalences.values.round(2)})
df.columns.name = 'Behavior problem'
display(df)
#exportTable(df, 'table_2') # exporting will overwrite custom format

In [ ]:
### Prevalence ranked by age of onset:

In [ ]:
# Create the necessary dataframe.
fields = []
labels = []
for counter, cat in enumerate(CATEGORY, 1):
    if counter > 10:
        counter += 1;
    fields.append('q02_main_{}'.format(counter))
    labels.append(cat)   
fields.append('q01_age_months')
labels.append('Age')
fields.append('q01_acq')
labels.append('Evident at Acquisition')
fields = ', '.join(fields)
df = createNumericDataFrame(DOG_TABLE, fields, labels, filtered=True)
#df = df[np.isfinite(df[labels[12]])]

# Ranges: 0=0-3m, 1=3-6m, 2=6m-1y, 3=1-3y, 4=3y+, 5=evident at acquisition
rngs = []
rngs.append(df[(df['Age'] < 3)].drop(columns=['Age', 'Evident at Acquisition']))
rngs.append(df[(df['Age'] >= 3) & (df['Age'] < 6)].drop(columns=['Age', 'Evident at Acquisition']))
rngs.append(df[(df['Age'] >= 6) & (df['Age'] < 12)].drop(columns=['Age', 'Evident at Acquisition']))
rngs.append(df[(df['Age'] >= 12) & (df['Age'] < 36)].drop(columns=['Age', 'Evident at Acquisition']))
rngs.append(df[(df['Age'] >= 36)].drop(columns=['Age', 'Evident at Acquisition']))
rngs.append(df[(df['Evident at Acquisition'] == 1)].drop(columns=['Age', 'Evident at Acquisition']))

def get_group_prevalence(df):
    df = df.sum().apply(lambda x: (x / len(df.index)) * 100)
    return df.round(2)

prevs = []
prevs.append(get_group_prevalence(rngs[0]))
prevs.append(get_group_prevalence(rngs[1]))
prevs.append(get_group_prevalence(rngs[2]))
prevs.append(get_group_prevalence(rngs[3]))
prevs.append(get_group_prevalence(rngs[4]))
prevs.append(get_group_prevalence(rngs[5]))

df = pd.DataFrame(index=prevs[0].index, data={'0-3m': prevs[0], '3-6m': prevs[1], '6-12m': prevs[2],
                                              '12-36m': prevs[3], '+36m': prevs[4],
                                              'Evident at Acquisition': prevs[5]})
df.columns.name = 'Behavior problem'
display(df)
#exportTable(df, 'table_4') # Exporting will overwrite custom formatting

In [ ]:
### Prevalence by gender and neutered status:

In [ ]:
# Create the necessary dataframe.
fields = []
labels = []
for counter, cat in enumerate(CATEGORY, 1):
    if counter > 10:
        counter += 1;
    fields.append('q02_main_{}'.format(counter))
    labels.append(cat)   
fields.extend(('dog_sex', 'dog_spayed'))
labels.extend(('Gender', 'Neutered'))
fields = ', '.join(fields)
df = createNumericDataFrame(DOG_TABLE, fields, labels, filtered=True)

df_intact_male = df[(df['Gender'] == 1) & (df['Neutered'] == 0)].drop(
    columns=['Gender', 'Neutered'])
df_neutered_male = df[(df['Gender'] == 1) & (df['Neutered'] == 1)].drop(
    columns=['Gender', 'Neutered'])
df_intact_female = df[(df['Gender'] == 2) & (df['Neutered'] == 0)].drop(
    columns=['Gender', 'Neutered'])
df_neutered_female = df[(df['Gender'] == 2) & (df['Neutered'] == 1)].drop(
    columns=['Gender', 'Neutered'])

def get_group_prevalence(df):
    df = df.sum().apply(lambda x: (x / len(df.index)) * 100)
    return df.round(2)

intact_male_p = get_group_prevalence(df_intact_male)
neutered_male_p = get_group_prevalence(df_neutered_male)
intact_female_p = get_group_prevalence(df_intact_female)
neutered_female_p = get_group_prevalence(df_neutered_female)

df = pd.DataFrame(index=intact_male_p.index, data={'Intact males': intact_male_p,
                                                   'Castrated males': neutered_male_p,
                                                   'Intact females': intact_female_p,
                                                   'Spayed females': neutered_female_p})
df.columns.name = 'Behavior problem'
display(df)
#exportTable(df, 'table_3') # Exporting will overwrite custom formatting

In [ ]:
### Prevalence by origin:

In [ ]:
# Create the necessary dataframe.
fields = []
labels = []
for counter, cat in enumerate(CATEGORY, 1):
    if counter > 10:
        counter += 1;
    fields.append('q02_main_{}'.format(counter))
    labels.append(cat)   
fields.append('acquisition_source')
labels.append('origin')
fields = ', '.join(fields)
df = createNumericDataFrame(DOG_TABLE, fields, labels, filtered=True)
# Rescue fields (online: 1, in-person: 4) were combined in the database.
#replacements = {'':'no response', '1': 'rescue', '2': 'online (non-rescue)', '3': 'pet store', '5': 'breeder',
#                '6': 'self-bred', '7': 'friends/family', '8': 'found', '9': 'other'}

rngs = []
for i in range(1, 9):
    cnt = i
    if i >= 4:
        cnt += 1
    rngs.append(df[(df['origin'] == cnt)].drop(columns=['origin']))

def get_group_prevalence(df):
    df = df.sum().apply(lambda x: (x / len(df.index)) * 100)
    return df.round(2)

prevs = []
for j in range(0, 8):
    prevs.append(get_group_prevalence(rngs[j]))

df = pd.DataFrame(index=intact_male_p.index, data={'Rescue': prevs[0], 'Online': prevs[1],
                                                   'Pet store': prevs[2], 'Breeder': prevs[3],
                                                   'Self-bred': prevs[4], 'Family/Friends': prevs[5],
                                                   'Found': prevs[6], 'Other': prevs[7]})
df.columns.name = 'Behavior problem'
display(df)
#exportTable(df, 'table_5') # Exporting will overwrite custom formatting

In [ ]:
### Prevalence by purebred lineage:

In [ ]:
# Create the necessary dataframe.
fields = []
labels = []
for counter, cat in enumerate(CATEGORY, 1):
    if counter > 10:
        counter += 1;
    fields.append('q02_main_{}'.format(counter))
    labels.append(cat)
fields.append('purebred')
labels.append('purebred')
fields = ', '.join(fields)
df = createNumericDataFrame(DOG_TABLE, fields, labels, filtered=True)

rngs = []
rngs.append(df[(df['purebred'] == 1)].drop(columns=['purebred']))
rngs.append(df[(df['purebred'] == 0)].drop(columns=['purebred']))

def get_group_prevalence(df):
    df = df.sum().apply(lambda x: (x / len(df.index)) * 100)
    return df.round(2)

prevs = []
prevs.append(get_group_prevalence(rngs[0]))
prevs.append(get_group_prevalence(rngs[1]))

df = pd.DataFrame(index=intact_male_p.index, data={'Purebred': prevs[0], 'Non-purebred': prevs[1]})
df.columns.name = 'Behavior problem'
display(df)
#exportTable(df, 'table_6') # Exporting will overwrite custom formatting

In [ ]:
### Impact of gender on behavior problem prevalence:

In [ ]:
fields = 'q02_score, dog_sex'
labels = ['behavior problems', 'gender']
df = createStringDataFrame(DOG_TABLE, fields, labels)
df = df[df[labels[1]] != '']
df = df.apply(pd.to_numeric)

def convert_to_binary_response(x):
    x = int(x)
    if x < 1:
        return 0
    return 1

def gender_to_binary_response(x):
    x = int(x)
    if x != 1:
        return 0
    return 1

df['behavior problems'] = df['behavior problems'].apply(
    lambda x: convert_to_binary_response(x))
df['gender'] = df['gender'].apply(
    lambda x: gender_to_binary_response(x))
contingency = pd.crosstab(df[labels[0]], df[labels[1]], margins=False)
display(contingency)

# Execute a chi-squared test of independence.
print('Chi-squared Test of Independence for %s and %s:' %(labels[0], labels[1]))
c, p, dof, expected = scs.chi2_contingency(contingency, correction=False)
print('chi2 = %f, p = %.2E, dof = %d' %(c, p, dof))
checkSignificance(p)
display(df.corr())

In [ ]:
### Impact of neutered status on behavior problem prevalence:

In [ ]:
fields = 'q02_score, dog_spayed'
labels = ['behavior problems', 'neutered']
df = createStringDataFrame(DOG_TABLE, fields, labels)
df = df[df[labels[1]] != '']
df = df[df[labels[1]] != '2']
df = df.apply(pd.to_numeric)

def convert_to_binary_response(x):
    x = int(x)
    if x < 1:
        return 0
    return 1

df['behavior problems'] = df['behavior problems'].apply(
    lambda x: convert_to_binary_response(x))
df['neutered'] = df['neutered'].apply(
    lambda x: convert_to_binary_response(x))
contingency = pd.crosstab(df[labels[0]], df[labels[1]], margins=False)
display(contingency)

# Execute a chi-squared test of independence.
print('Chi-squared Test of Independence for %s and %s:' %(labels[0], labels[1]))
c, p, dof, expected = scs.chi2_contingency(contingency, correction=False)
print('chi2 = %f, p = %.2E, dof = %d' %(c, p, dof))
checkSignificance(p)
display(df.corr())

In [ ]:
### Impact of source of origin on behavior problem prevalence:

In [ ]:
fields = 'q02_score, acquisition_source'
labels = ['behavior problems', 'origin']
df = createStringDataFrame(DOG_TABLE, fields, labels)
# Rescue fields (online: 1, in-person: 4) were combined in the database.
#replacements = {'':'no response', '1': 'rescue', '2': 'online (non-rescue)', '3': 'pet store', '5': 'breeder',
#                '6': 'self-bred', '7': 'friends/family', '8': 'found', '9': 'other'}
df = df[df[labels[1]] != '']
df = df.apply(pd.to_numeric)

def convert_to_binary_response(x):
    x = int(x)
    if x < 1:
        return 0
    return 1

def rescue_to_binary_response(x):
    x = int(x)
    if x != 1:
        return 0
    return 1

df['behavior problems'] = df['behavior problems'].apply(
    lambda x: convert_to_binary_response(x))
df['origin'] = df['origin'].apply(lambda x: rescue_to_binary_response(x))
contingency = pd.crosstab(df[labels[0]], df[labels[1]], margins=False)
display(contingency)

# Execute a chi-squared test of independence.
print('Chi-squared Test of Independence for %s and %s:' %(labels[0], labels[1]))
c, p, dof, expected = scs.chi2_contingency(contingency, correction=False)
print('chi2 = %f, p = %.2E, dof = %d' %(c, p, dof))
checkSignificance(p)
display(df.corr())

In [ ]:
### Impact of purebred status on behavior problem prevalence:

In [ ]:
fields = 'q02_score, purebred'
labels = ['behavior problems', 'purebred']
df = createStringDataFrame(DOG_TABLE, fields, labels)
df = df[df[labels[1]] != '']
df = df.apply(pd.to_numeric)

def convert_to_binary_response(x):
    x = int(x)
    if x < 1:
        return 0
    return 1

df['behavior problems'] = df['behavior problems'].apply(
    lambda x: convert_to_binary_response(x))
contingency = pd.crosstab(df[labels[0]], df[labels[1]], margins=False)
display(contingency)

# Execute a chi-squared test of independence.
print('Chi-squared Test of Independence for %s and %s:' %(labels[0], labels[1]))
c, p, dof, expected = scs.chi2_contingency(contingency, correction=False)
print('chi2 = %f, p = %.2E, dof = %d' %(c, p, dof))
checkSignificance(p)
display(df.corr())

In [ ]:
### Impact of age on prevalence:

In [ ]:
fields = 'q02_score, q01_age_months'
labels = ['behavior problems', 'age']
df = createStringDataFrame(DOG_TABLE, fields, labels)
df = df[df[labels[1]] != '']
df = df.apply(pd.to_numeric)
display(df.describe())

def convert_to_binary_response(x, y):
    x = float(x)
    if x < y:
        return 0
    return 1

df['age'] = df['age'].apply(lambda x: convert_to_binary_response(x, 12))
df['behavior problems'] = df['behavior problems'].apply(lambda x: convert_to_binary_response(x, 1))
contingency = pd.crosstab(df[labels[0]], df[labels[1]], margins=False)
display(contingency)

# Execute a chi-squared test of independence.
print('Chi-squared Test of Independence for %s and %s:' %(labels[0], labels[1]))
c, p, dof, expected = scs.chi2_contingency(contingency, correction=False)
print('chi2 = %f, p = %.2E, dof = %d' %(c, p, dof))
checkSignificance(p)
display(df.corr())

## <center>Number of Behavior Problems</center>

### Number of behavior problems per dog:

In [ ]:
fields = 'q02_score'
labels = ['number of behavior problems']
df = createNumericDataFrame(DOG_TABLE, fields, labels)
display(df.describe())

df = df.apply(pd.value_counts)
df.rename(index=str, columns={labels[0]: "frequency"}, inplace=True)
df.columns.name = labels[0]
display(df)

In [ ]:
### Number of behavior problems by age range:

In [ ]:
# Create the necessary dataframe.
fields = ['q02_score', 'q01_age_months']
labels = ['behavior problems', 'age'] 
fields = ', '.join(fields)
df = createNumericDataFrame(DOG_TABLE, fields, labels, filtered=True)

# Ranges: 0=0-3m, 1=3-6m, 2=6m-1y, 3=1-3y, 4=3y+, 5=evident at acquisition
rngs = []
rngs.append(df[(df['age'] < 3)].drop(columns=['age']))
rngs.append(df[(df['age'] >= 3) & (df['age'] < 6)].drop(columns=['age']))
rngs.append(df[(df['age'] >= 6) & (df['age'] < 12)].drop(columns=['age']))
rngs.append(df[(df['age'] >= 12) & (df['age'] < 36)].drop(columns=['age']))
rngs.append(df[(df['age'] >= 36)].drop(columns=['age']))

def get_group_mean(df):
    return float(df.mean())

means = []
for i, val in enumerate(rngs):
    means.append(get_group_mean(rngs[i]))
    
df = pd.DataFrame(index=['0-3m', '3-6m', '6-12m', '12-36m', '+36m'], data={'Number of Behavior Problems': means})
df.columns.name = 'Age Range'
display(df)
exportTable(df, 'table_7') # Exporting will overwrite custom formatting

In [ ]:
### Impact of gender on number of behavior problems:

In [ ]:
fields = 'q02_score, dog_sex'
labels = ['behavior problems', 'gender']
df = createStringDataFrame(DOG_TABLE, fields, labels)
df = df[df[labels[1]] != '']
df = df.apply(pd.to_numeric)

males = df[(df['gender'] == 1)].drop(columns=['gender'])
display(males.describe())
females = df[(df['gender'] == 2)].drop(columns=['gender'])
display(females.describe())
ttest_ind(males, females, equal_var=False)

In [ ]:
### Impact of neuter status on number of behavior problems:

In [ ]:
fields = 'q02_score, dog_spayed'
labels = ['behavior problems', 'neutered']
df = createStringDataFrame(DOG_TABLE, fields, labels)
df = df[df[labels[1]] != '']
df = df.apply(pd.to_numeric)

neutered = df[(df['neutered'] == 1)].drop(columns=['neutered'])
display(neutered.describe())
intact = df[(df['neutered'] == 0)].drop(columns=['neutered'])
display(intact.describe())
ttest_ind(neutered, intact, equal_var=False)

In [ ]:
### Impact of purebred lineage on number of behavior problems:

In [ ]:
fields = 'q02_score, purebred'
labels = ['behavior problems', 'purebred']
df = createStringDataFrame(DOG_TABLE, fields, labels)
df = df[df[labels[1]] != '']
df = df.apply(pd.to_numeric)


purebred = df[(df['purebred'] == 1)].drop(columns=['purebred'])
display(purebred.describe())
notpure = df[(df['purebred'] == 0)].drop(columns=['purebred'])
display(notpure.describe())
ttest_ind(notpure, purebred, equal_var=False)

In [ ]:
### Impact of acquisition source on number of behavior problems:

In [ ]:
fields = 'q02_score, acquisition_source'
labels = ['behavior problems', 'origin']
df = createStringDataFrame(DOG_TABLE, fields, labels)
df = df[df[labels[1]] != '']
df = df.apply(pd.to_numeric)

def rescue_to_binary_response(x):
    x = int(x)
    if x != 1:
        return 0
    return 1

df['origin'] = df['origin'].apply(lambda x: rescue_to_binary_response(x))
rescue = df[(df['origin'] == 1)].drop(columns=['origin'])
display(rescue.describe())
nonrescue = df[(df['origin'] == 0)].drop(columns=['origin'])
display(nonrescue.describe())
ttest_ind(rescue, nonrescue, equal_var=False)

## <center>Separation Anxiety, Noise Phobia, and Thunderstorm Phobia</center>

In [ ]:
### Paired independence:

In [ ]:
# Create the necessary dataframe.
fields = 'q04_1, q04_2, q04_9'
labels = ['Thunderstorm phobia', 'Noise phobia', 'Separation anxiety']
df = createNumericDataFrame(DOG_TABLE, fields, labels, filtered=True)

pairs = [[labels[0], labels[1]], [labels[0], labels[2]], [labels[1], labels[2]]]
for pair in pairs:
    # Create a contingency table.
    contingency = pd.crosstab(df[pair[0]], df[pair[1]])
    # Execute a chi-squared test of independence.
    print('Chi-squared Test of Independence for %s and %s:' %(pair[0], pair[1]))
    c, p, dof, expected = scs.chi2_contingency(contingency, correction=False)
    display(contingency)
    displayOddsRatio(contingency)
    print('chi2 = %f, p = %.2E, dof = %d' %(c, p, dof))
    checkSignificance(p)
    print()

In [ ]:
### Grouped independence:

In [ ]:
# Create a contingency table.
labels = ['Thunderstorm phobia', 'Noise phobia', 'Separation anxiety']
contingency = pd.crosstab(df[labels[2]], [df[labels[0]], df[labels[1]]])

# Convert the cross tabulated dataframe to a series of sums.
d = {labels[2]: contingency[0][0][1],
     labels[1]: contingency[1][0][0],
     'Separation-Noise': contingency[1][0][1],
     labels[0]: contingency[0][1][0],
     'Separation-Thunderstorm': contingency[0][1][1],
     'Noise-Thunderstorm': contingency[1][1][0],     
     'All': contingency[1][1][1]}   
cross_sums = pd.Series(d)

# Display the cross tabulated data as a venn diagram.
v = venn3(cross_sums, set_labels=[cross_sums.index.values[i] for i in [0, 1, 3]])
lbl = v.get_label_by_id('A')
x, y = lbl.get_position()
lbl.set_position((x+0.25, y+0.05))
for text in v.set_labels:
    text.set_fontsize(16)
for text in v.subset_labels:
    text.set_fontsize(14)
exportFigure(plt, 'figure_1')
plt.show()

# Execute a chi-squared test of independence.
print('Chi-squared Test of Independence for %s, %s, and %s:' %(labels[0], labels[1], labels[2]))
c, p, dof, expected = scs.chi2_contingency(contingency, correction=False)
print('chi2 = %f, p = %.2E, dof = %d' %(c, p, dof))
checkSignificance(p)
display(df.corr())

## <center>Separation Anxiety, Destructive Behavior, and House Soiling</center>

In [ ]:
### Desctructive and separation anxiety:

In [ ]:
# Create the necessary dataframe.
fields = 'q02_main_9, q04_9'
labels = ['Destruction', 'Separation anxiety']
df = createNumericDataFrame(DOG_TABLE, fields, labels, filtered=True)

# Cross tabulate the relevant columns.
contingency = pd.crosstab(df[labels[0]], df[labels[1]], margins=False)
display(contingency)
displayOddsRatio(contingency)

# Convert the cross tabulated dataframe to a series of sums.
d = {labels[0]: contingency[0][1],
     labels[1]: contingency[1][0],
     'Both': contingency[1][1]}   
cross_sums = pd.Series(d)

# Display the cross tabulated data as a venn diagram.
labels = cross_sums.index.values
v = venn2(cross_sums, set_labels=labels[0:2])
lbl = v.get_label_by_id('B')
x, y = lbl.get_position()
lbl.set_position((x+0.15, y+0.05))
for text in v.set_labels:
    text.set_fontsize(16)
for text in v.subset_labels:
    text.set_fontsize(14)
exportFigure(plt, 'figure_5')
plt.show()

# Execute a chi-squared test of independence.
c, p, dof, expected = scs.chi2_contingency(contingency, correction=False)
print('Chi-square Test of Independence:')
print('chi2 = %f, p = %.2E, dof = %d' %(c, p, dof))
checkSignificance(p)
display(df.corr())

In [ ]:
### House soiling during owner absence and separation anxiety:

In [ ]:
# Create the necessary dataframe.
fields = 'q06_situation_2, q04_9'
labels = ['House soiling (owner absence)', 'Separation anxiety']
df = createNumericDataFrame(DOG_TABLE, fields, labels, filtered=True)

# Cross tabulate the relevant columns.
contingency = pd.crosstab(df[labels[0]], df[labels[1]], margins=False)
display(contingency)
displayOddsRatio(contingency)

# Convert the cross tabulated dataframe to a series of sums.
d = {labels[0]: contingency[0][1],
     labels[1]: contingency[1][0],
     'Both': contingency[1][1]}   
cross_sums = pd.Series(d)

# Display the cross tabulated data as a venn diagram.
labels = cross_sums.index.values
v = venn2(cross_sums, set_labels=labels[0:2])
lbl = v.get_label_by_id('B')
x, y = lbl.get_position()
lbl.set_position((x+0.15, y+0.05))
for text in v.set_labels:
    text.set_fontsize(16)
for text in v.subset_labels:
    text.set_fontsize(14)
exportFigure(plt, 'figure_5')
plt.show()

# Execute a chi-squared test of independence.
c, p, dof, expected = scs.chi2_contingency(contingency, correction=False)
print('Chi-square Test of Independence:')
print('chi2 = %f, p = %.2E, dof = %d' %(c, p, dof))
checkSignificance(p)
display(df.corr())

## <center>Fear/Anxiety and House Soiling</center>

## <center>Fear/Anxiety and Aggression</center>

In [ ]:
### Overall aggression and fearful/anxious behavior:

In [ ]:
# Create the necessary dataframe.
fields = 'q02_main_1, q02_main_2'
labels = ['Aggression', 'Fear/Anxiety']
df = createNumericDataFrame(DOG_TABLE, fields, labels, filtered=True)
    
# Create a contingency table.
contingency = pd.crosstab(df[labels[0]], df[labels[1]], margins=False)
display(contingency)
displayOddsRatio(contingency)

# Convert the cross tabulated dataframe to a series of sums.
d = {labels[0]: contingency[0][1],
     labels[1]: contingency[1][0],
     'Both': contingency[1][1]}   
cross_sums = pd.Series(d)

# Display the cross tabulated data as a venn diagram.
labels = cross_sums.index.values
v = venn2(cross_sums, set_labels=labels[0:2])
lbl = v.get_label_by_id('A')
x, y = lbl.get_position()
lbl.set_position((x-0.1, y+0.05))
for text in v.set_labels:
    text.set_fontsize(16)
for text in v.subset_labels:
    text.set_fontsize(14)
exportFigure(plt, 'figure_2')
plt.show()

# Execute a chi-squared test of independence.
c, p, dof, expected = scs.chi2_contingency(contingency, correction=False)
print('Chi-square Test of Independence:')
print('chi2 = %f, p = %.2E, dof = %d' %(c, p, dof))
checkSignificance(p)
display(df.corr())

In [ ]:
### Owner directed aggression and fearful/anxious behavior:

In [ ]:
# Create the necessary dataframe.
fields = 'q03_main_1, q02_main_2'
labels = ['Owner directed\naggression', 'Fear/Anxiety']
df = createNumericDataFrame(DOG_TABLE, fields, labels, filtered=True)
    
# Create a contingency table.
contingency = pd.crosstab(df[labels[0]], df[labels[1]], margins=False)
display(contingency)
displayOddsRatio(contingency)

# Convert the cross tabulated dataframe to a series of sums.
d = {labels[0]: contingency[0][1],
     labels[1]: contingency[1][0],
     'Both': contingency[1][1]}   
cross_sums = pd.Series(d)

# Display the cross tabulated data as a venn diagram.
labels = cross_sums.index.values
v = venn2(cross_sums, set_labels=labels[0:2])
lbl = v.get_label_by_id('A')
x, y = lbl.get_position()
lbl.set_position((x-0.1, y+0.05))
for text in v.set_labels:
    text.set_fontsize(16)
for text in v.subset_labels:
    text.set_fontsize(14)
exportFigure(plt, 'figure_3')
plt.show()

# Execute a chi-squared test of independence.
c, p, dof, expected = scs.chi2_contingency(contingency, correction=False)
print('Chi-square Test of Independence:')
print('chi2 = %f, p = %.2E, dof = %d' %(c, p, dof))
checkSignificance(p)
display(df.corr())

In [ ]:
### Owner directed aggression and separation anxiety:

In [ ]:
# Create the necessary dataframe.
fields = 'q03_main_1, q04_9'
labels = ['Owner directed\naggression', 'Separation anxiety']
df = createNumericDataFrame(DOG_TABLE, fields, labels, filtered=True)
    
# Create a contingency table.
contingency = pd.crosstab(df[labels[0]], df[labels[1]], margins=False)
display(contingency)
displayOddsRatio(contingency)

# Convert the cross tabulated dataframe to a series of sums.
d = {labels[0]: contingency[0][1],
     labels[1]: contingency[1][0],
     'Both': contingency[1][1]}   
cross_sums = pd.Series(d)

# Display the cross tabulated data as a venn diagram.
labels = cross_sums.index.values
v = venn2(cross_sums, set_labels=labels[0:2])
for text in v.set_labels:
    text.set_fontsize(16)
for text in v.subset_labels:
    text.set_fontsize(14)
exportFigure(plt, 'figure_4')
plt.show()

# Execute a chi-squared test of independence.
c, p, dof, expected = scs.chi2_contingency(contingency, correction=False)
print('Chi-square Test of Independence:')
print('chi2 = %f, p = %.2E, dof = %d' %(c, p, dof))
checkSignificance(p)
display(df.corr())

## <center>Fear/Anxiety and Compulsive Behavior</center>

In [ ]:
### (Figure ?)

In [ ]:
# Create the necessary dataframe.
fields = 'q02_main_2, q02_main_3'
labels = ['Fear/Anxiety', 'Compulsion']
df = createNumericDataFrame(DOG_TABLE, fields, labels, filtered=True)

# Cross tabulate the relevant columns.
contingency = pd.crosstab(df[labels[0]], df[labels[1]], margins=False)
display(contingency)
displayOddsRatio(contingency)

# Convert the cross tabulated dataframe to a series of sums.
d = {labels[0]: contingency[0][1],
     labels[1]: contingency[1][0],
     'Both': contingency[1][1]}   
cross_sums = pd.Series(d)

# Display the cross tabulated data as a venn diagram.
labels = cross_sums.index.values
v = venn2(cross_sums, set_labels=labels[0:2])
lbl = v.get_label_by_id('B')
x, y = lbl.get_position()
lbl.set_position((x+0.15, y+0.05))
for text in v.set_labels:
    text.set_fontsize(16)
for text in v.subset_labels:
    text.set_fontsize(14)
exportFigure(plt, 'figure_5')
plt.show()

# Execute a chi-squared test of independence.
c, p, dof, expected = scs.chi2_contingency(contingency, correction=False)
print('Chi-square Test of Independence:')
print('chi2 = %f, p = %.2E, dof = %d' %(c, p, dof))
checkSignificance(p)
display(df.corr())

## <center>Coprophagia and Age</center>

In [ ]:
fields = ', '.join(FR.fields[CATEGORY[7]])
fields += ', q01_age_months'
labels = list(FR.labels[CATEGORY[7]].values())
labels.append('age')
df = createNumericDataFrame(DOG_TABLE, fields, labels, filtered=True)


# Ranges: 0=0-3m, 1=3-6m, 2=6m-1y, 3=1-3y, 4=3y+, 5=evident at acquisition
df['age<1y'] = np.where(df['age'] < 12, 1, 0)
df['age>=1y'] = np.where(df['age'] >= 12, 1, 0)
df = df.fillna(0)
df['NR'] = np.where(df['age'] == 0, 1, 0)
df = df.drop(columns=['age'])

df_copro = df[(df[labels[0]] == 1) | (df[labels[1]] == 1) | (df[labels[2]] == 1)].drop(
    columns=[labels[0], labels[1], labels[2]])

def get_group_prevalence(df):
    df = df.sum().apply(lambda x: (x / 1) * 100)
    return df.round(2)

df = df_copro.sum()
display(df_copro.sum())

print("These results are no good!")

# Manually I got 605 with age < 12 months and 112 with age > 12 months.

## <center>Overactivity/Hyperactivity and Age</center>

In [ ]:
# Done by hand
# SELECT COUNT(*) FROM dogs JOIN users USING (record_id) WHERE (question_reason_for_part_3 = 0 OR (question_reason_for_part_3 = 1 AND q01_main != 1)) AND (q01_age_months > 12 and (q02_main_13));
# age < 12 months = 409, age > 12 months = 96 --> total = 505

## <center>Bite Severity</center>

In [ ]:
### Breakdown of bite severity:

In [ ]:
fields = 'q03_severity'
labels = ['severity']
df = createNumericDataFrame(DOG_TABLE, fields, labels)
# Rescue fields (online: 1, in-person: 4) were combined in the database.
#replacements = {'':'no response', '1': 'rescue', '2': 'online (non-rescue)', '3': 'pet store', '5': 'breeder',
#                '6': 'self-bred', '7': 'friends/family', '8': 'found', '9': 'other'}
#replaceFields(df, labels[0], replacements)
df = df.apply(pd.value_counts)
df.rename(index=str, columns={labels[0]: "count"}, inplace=True)
df.columns.name = labels[0]
display(df)

In [ ]:
### Bite severity by behavior problem:

In [ ]:
# Create the necessary dataframe.
fields = []
labels = []
for counter, cat in enumerate(CATEGORY, 1):
    if counter > 10:
        counter += 1;
    fields.append('q02_main_{}'.format(counter))
    labels.append(cat)
fields.append('q03_severity')
labels.append('severity')
fields = ', '.join(fields)
df = createNumericDataFrame(DOG_TABLE, fields, labels, filtered=True)


rngs = []
rngs.append(df[(df['severity'] == 1)].drop(columns=['severity']))
rngs.append(df[(df['severity'] == 2)].drop(columns=['severity']))
rngs.append(df[(df['severity'] == 3)].drop(columns=['severity']))
rngs.append(df[(df['severity'] == 4)].drop(columns=['severity']))
rngs.append(df[(df['severity'] == 5)].drop(columns=['severity']))

def get_group_prevalence(df):
    df = df.sum().apply(lambda x: (x / len(df.index)) * 100)
    #df = df.sum().apply(lambda x: (x / 1))
    return df.round(2)

prevs = []
prevs.append(get_group_prevalence(rngs[0]))
prevs.append(get_group_prevalence(rngs[1]))
prevs.append(get_group_prevalence(rngs[2]))
prevs.append(get_group_prevalence(rngs[3]))
prevs.append(get_group_prevalence(rngs[4]))

df = pd.DataFrame(index=prevs[0].index, data={'level 1': prevs[0], 'level 2': prevs[1], 'level 3': prevs[2],
                                                   'level 4': prevs[3], 'level 5': prevs[4]})
df.columns.name = 'Behavior problem'
display(df)
#exportTable(df, 'table_6') # Exporting will overwrite custom formatting

In [ ]:
### Bite severity and fear/anxiety:

In [ ]:
# Create the necessary dataframe.
fields = ', '.join(FR.fields[CATEGORY[1]])
labels = list(FR.labels[CATEGORY[1]].values())
fields += ', q03_severity'
labels.append('severity')
df = createNumericDataFrame(DOG_TABLE, fields, labels, filtered=True)

rngs = []
rngs.append(df[(df['severity'] == 1)].drop(columns=['severity']))
rngs.append(df[(df['severity'] == 2)].drop(columns=['severity']))
rngs.append(df[(df['severity'] == 3)].drop(columns=['severity']))
rngs.append(df[(df['severity'] == 4)].drop(columns=['severity']))
rngs.append(df[(df['severity'] == 5)].drop(columns=['severity']))

def get_group_prevalence(df):
    df = df.sum().apply(lambda x: (x / len(df.index)) * 100)
    return df.round(2)

prevs = []
prevs.append(get_group_prevalence(rngs[0]))
prevs.append(get_group_prevalence(rngs[1]))
prevs.append(get_group_prevalence(rngs[2]))
prevs.append(get_group_prevalence(rngs[3]))
prevs.append(get_group_prevalence(rngs[4]))

df = pd.DataFrame(index=prevs[0].index, data={'level 1': prevs[0], 'level 2': prevs[1], 'level 3': prevs[2],
                                                   'level 4': prevs[3], 'level 5': prevs[4]})
df.columns.name = 'Behavior problem'
display(df)
#exportTable(df, 'table_6') # Exporting will overwrite custom formatting

In [ ]:
### Bite severity and gender and neutered status

In [ ]:
#Construct the dataframe.
fields = ['q03_severity']
labels = ['severity']
fields.extend(('dog_sex', 'dog_spayed'))
labels.extend(('Gender', 'Neutered'))
fields = ', '.join(fields)
df = createNumericDataFrame(DOG_TABLE, fields, labels, filtered=True)

df['level 1'] = np.where(df['severity'] == 1, 1, 0)
df['level 2'] = np.where(df['severity'] == 2, 1, 0)
df['level 3'] = np.where(df['severity'] == 3, 1, 0)
df['level 4'] = np.where(df['severity'] == 4, 1, 0)
df['level 5'] = np.where(df['severity'] == 5, 1, 0)
df = df.drop(columns=['severity'])


df_intact_male = df[(df['Gender'] == 1) & (df['Neutered'] == 0)].drop(
    columns=['Gender', 'Neutered'])
df_neutered_male = df[(df['Gender'] == 1) & (df['Neutered'] == 1)].drop(
    columns=['Gender', 'Neutered'])
df_intact_female = df[(df['Gender'] == 2) & (df['Neutered'] == 0)].drop(
    columns=['Gender', 'Neutered'])
df_neutered_female = df[(df['Gender'] == 2) & (df['Neutered'] == 1)].drop(
    columns=['Gender', 'Neutered'])

def get_group_prevalence(df):
    #df = df.sum().apply(lambda x: (x / len(df.index)) * 100)
    df = df.sum().apply(lambda x: (x / 1))
    return df.round(2)

intact_male_p = get_group_prevalence(df_intact_male)
neutered_male_p = get_group_prevalence(df_neutered_male)
intact_female_p = get_group_prevalence(df_intact_female)
neutered_female_p = get_group_prevalence(df_neutered_female)

df = pd.DataFrame(index=intact_male_p.index, data={'Intact males': intact_male_p,
                                                   'Castrated males': neutered_male_p,
                                                   'Intact females': intact_female_p,
                                                   'Spayed females': neutered_female_p})
df.columns.name = 'Behavior problem'
display(df)

# by hand: 3.504043127	52.83018868	3.234501348	40.43126685	100

## <center>Appendix</center>

In [ ]:
### Behavior problem category heat map (Appendix 2)

In [ ]:
if CATEGORY_MATRIX.empty:
    CATEGORY_MATRIX = createCategoryMatrix()
    
# Create a new dataframe with categories:
df = CATEGORY_MATRIX.applymap(lambda row: get_significance_category(row))
df_or = createOddsRatioMatrix()

# Heatmap
fig, ax = plt.subplots(figsize=(20, 15))
cmap = sns.cubehelix_palette(n_colors=3)
ax = sns.heatmap(data=df, ax=ax, cmap=ListedColormap(cmap), linewidths=0.5, linecolor='black',
                 cbar=False, mask=df.isnull(), annot=df_or, annot_kws={"size": 28}, fmt='.2f')
plt.xticks(rotation=0, fontsize=20)
plt.yticks(rotation=0, fontsize=20)
ax.invert_yaxis()
import matplotlib.patches as mpatches
patch_1 = mpatches.Patch(color=cmap[0], label='$p\geq10^{-3}$')
patch_2 = mpatches.Patch(color=cmap[1], label='$10^{-3}>p\geq10^{-6}$')
patch_3 = mpatches.Patch(color=cmap[2], label='$10^{-6}>p$')
legend = plt.legend(handles=[patch_1, patch_2, patch_3], bbox_to_anchor=(0.5, -0.05), loc='upper center',
                    fontsize=28, ncol=3)
legend.get_frame().set_edgecolor('black')
exportFigure(fig, 'figure_6')
plt.show()

In [ ]:
### Heat map axes abbrevation reference chart (Appendix 3)

In [ ]:
# Construct lists.
cat_dict = {}
for cat, subdict in FR.labels.items():
    for key, value in  subdict.items():
        cat_dict[key[0]] = cat
        break

s = pd.Series(cat_dict)
df = pd.DataFrame(index=s.index, data={'Category':s.values})
df.columns.name = 'Index'
display(df)
exportTable(df, 'table_category_reference')